In [ ]:
import os
import numpy as np
import plotly.io as pio
from matplotlib import pyplot as plt

import prepare_data
import run_detection
import plot_detection

pio.renderers.default = 'vscode'
%matplotlib inline

# Data Inspection

## Prepare Data

Configure file paths

In [ ]:
DATA_DIR = 'assets/'

ALL_HE_DIR = DATA_DIR + 'All_He/'
SELECT_HE_DIR = DATA_DIR + 'Selected_He/'

IMG_EUV_DIR = DATA_DIR + 'Img_EUV/'

SINGLE_MAPS_DIR = DATA_DIR + 'Output/Single_Maps/'
ENSEMBLE_MAPS_DIR = DATA_DIR + 'Output/Ensemble_Maps_v0_3/'
ENSEMBLE_MAP_SAVE_DIR = ENSEMBLE_MAPS_DIR + 'Numpy_Arrays/'

Rename Data
- He I and magnetogram FITS files are renamed automatically

In [ ]:
# Delete all gzipped files after renaming
remove_gzip = True

# Rename all He FITS files to include observation date in title
prepare_data.rename_dir(ALL_HE_DIR, remove_gzip)

Specify Date Range
- WSA chs plots and EUV images must be renamed to match the string format DICT_DATE_STR_FORMAT

In [ ]:
# Option to extract all or only select few He FITS files of interest
# All dates
# date_range = ('2000_01_01__00_00', '2020_01_01__00_00')

# CR 2136
# date_range = ('2013_04_01__00_00', '2013_06_01__00_00')

# CR 2151
# date_range = ('2014_06_01__00_00', '2014_06_30__00_00')

# GMU period early in Solar Cycle 24
date_range = ('2012_04_01__00_00', '2012_09_01__00_00')

# Select few dates for analysis
# date_range = None

Extract Data from File System

In [ ]:
# Extract data arrays from He FITS files
HE_FITS_DICT = prepare_data.extract_he(date_range, ALL_HE_DIR, SELECT_HE_DIR)
HE_DATE_LIST = list(HE_FITS_DICT.keys())
HE_DATE_LIST.sort()

# Extract EUV images from image files
EUV_DICT = prepare_data.extract_comparison_ims(IMG_EUV_DIR)

## Available Data

In [ ]:
print('Available Datetimes for He I Observations:')
prepare_data.display_dates(HE_DATE_LIST)

He I FITS File Content

In [ ]:
date_str = '2015_03_31__18_13'
# date_str = '2013_04_17__18_39'
# date_str = '2014_06_06__17_46'

cmaps = [plt.cm.RdBu, plt.cm.afmhot, plt.cm.gray]

plot_detection.plot_fits_content(HE_FITS_DICT, date_str, cmaps)

# Pre-Process

In [ ]:
from skimage import exposure


def remove_peak_counts(array):
    """Retrieve an array with the value of peak counts replaced with NaN.
    """
    peak_counts_val = run_detection.get_peak_counts_loc(array, bins_as_percent=False)
    zero_vals = (array > peak_counts_val - 1e-2) & (array < peak_counts_val + 1e-2)
    
    return np.where(zero_vals, np.NaN, array)


def remove_background(array):
    """Retrieve an array with the background replaced with NaN.
    """
    background_val = array[0,0]    
    return np.where(array == background_val, np.NaN, array)


def pre_process_eqw_v0_4(raw_eqw):
    """Pre-process equivalent width array by setting background to NaN
    and a simple brightness band pass.
    """
    eqw_nan = np.where(raw_eqw == 0, np.NaN, raw_eqw)
    
    eqw_high_cut = np.where(eqw_nan > 100, np.NaN, eqw_nan)
    # eqw_band_cut = np.where(eqw_high_cut < -100, np.NaN, eqw_high_cut)
    eqw_band_cut = np.clip(eqw_high_cut, -100, 100)
    
    return eqw_band_cut, eqw_high_cut, eqw_nan


def equalize(raw_eqw):
    """Pre-process equivalent width array by setting background to NaN
    and a simple brightness band pass.
    """
    # Histogram equalization
    eqw1 = exposure.equalize_hist(raw_eqw)
    eqw1 = remove_background(eqw1)
    
    # Shift nonzero values into positive range and equalize histogram
    eqw2 = np.where(raw_eqw == 0, 0, raw_eqw + np.abs(np.min(raw_eqw)))
    eqw3 = exposure.equalize_hist(eqw2)
    
    eqw3 = np.where(eqw3 == np.min(eqw3), np.NaN, eqw3)
    
    return eqw3, eqw2, eqw1


def normalize(raw_eqw):
    """Pre-process equivalent width array by applying linear rescaling
    to normalize the contrast and setting background to NaN. Linear
    rescaling between 2-98 percentiles produces a less harsh contrast
    enhancement than histogram equalization.
    """
    p2, p98 = np.percentile(raw_eqw, (2, 98))
    
    # Shift nonzero values into positive range and normalize
    eqw1 = np.where(raw_eqw == 0, 0, raw_eqw + np.abs(np.min(raw_eqw)))
    eqw2 = exposure.rescale_intensity(eqw1, in_range=(p2, p98))
    
    # Normalize directly
    eqw3 = exposure.rescale_intensity(raw_eqw, in_range=(p2, p98))
    eqw3 = remove_background(eqw3)
        
    return eqw3, eqw2, eqw1

## Single Observation

In [ ]:
date_str = '2012_05_07__17_59'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw_band_cut, eqw_high_cut, eqw_nan = run_detection.pre_process_eqw_v0_1(
    raw_eqw, peak_count_cutoff_percent=0.1
)

arrays = [raw_eqw, eqw_nan, eqw_high_cut, eqw_band_cut]
titles = ['EQW', 'EQW NaN', 'EQW High Cut', 'EQW Band Cut']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

In [ ]:
date_str = '2012_05_07__17_59'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw_band_cut, eqw_high_cut, eqw_nan = pre_process_eqw_v0_3(raw_eqw)

arrays = [raw_eqw, eqw_nan, eqw_high_cut, eqw_band_cut]
titles = ['EQW', 'EQW NaN', 'EQW High Cut', 'EQW Band Cut']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

In [ ]:
date_str = '2012_05_07__17_59'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw3, eqw2, eqw1 = equalize(raw_eqw)

arrays = [raw_eqw, eqw1, eqw2, eqw3]
titles = ['EQW', 'Equalized', 'Shifted', 'Shifted & Equalized']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

Normalized

In [ ]:
date_str = '2012_05_07__17_59'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw3, eqw2, eqw1 = normalize(raw_eqw)

arrays = [raw_eqw, eqw3, eqw1, eqw2]
titles = ['EQW', 'Stretched', 'Shifted', 'Shifted & Stretched']

plot_detection.plot_hists(arrays[0:2], titles[0:2], semilogy=True)
plot_detection.plot_hists(arrays[2:4], titles[2:4], semilogy=True)

## Compare Observations

In [ ]:
date_idx = 48 # 32: 05/06-08   # 48: 05/26-28

for date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw_band_cut = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    arrays = [raw_eqw, eqw_band_cut]
    titles = [date_str, 'Pre-Processed']
    plot_detection.plot_hists(arrays, titles, semilogy=True)

In [ ]:
for date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw_band_cut = pre_process_eqw_v0_3(raw_eqw)[0]
    arrays = [raw_eqw, eqw_band_cut]
    titles = [date_str, 'Pre-Processed']
    plot_detection.plot_hists(arrays, titles, semilogy=True)

Normalized

In [ ]:
for date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw = normalize(raw_eqw)[0]
    arrays = [raw_eqw, eqw]
    titles = [date_str, 'Normalized']
    plot_detection.plot_hists(arrays, titles, semilogy=True)

# Parameters

## Threshold

Single Observation

In [ ]:
date_str = '2012_06_22__18_13'

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

plot_detection.plot_thresholds(eqw, bounds=[75, 85, 100], bounds_as_percent=True)

Compare Observations

In [ ]:
for date_str in HE_DATE_LIST[32:35]:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    plot_detection.plot_thresholds(eqw, bounds=[75, 85, 100], bounds_as_percent=True)

In [ ]:
date_str = '2015_03_31__18_13'
step = 2

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

percent_of_peak_lists = [list(np.arange(0,200,step)), list(np.arange(80,130,step))]

for percent_of_peak_list in percent_of_peak_lists:
    area_percent_list = run_detection.get_thresh_area_percent_list(eqw, percent_of_peak_list)
    
    parameter_stats = run_detection.get_parameter_stats(area_percent_list)
    max_diff, cutoff, selected_parameter_num, pixel_percent_diffs = parameter_stats
    
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot()

    plot_detection.plot_pixel_percent_bars(
        ax, percent_of_peak_list, area_percent_list, max_diff, cutoff, selected_parameter_num,
        step, title='Threshold', unit='%', xlabel='Percent of Peak Pixel Count (%)', thresh=True)

## Structuring Element Radius

In [ ]:
date_str = '2012_06_22__18_13'

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = pre_process_eqw_v0_3(raw_eqw)[0]

percent_of_peak_list = [90, 90, 90]
morph_radius_list = [4,10,14]

ch_mask_list = [
    run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]
title_list = [f'{percent_of_peak:d}% of Peak | {radius:d}px Radius'
              for percent_of_peak in percent_of_peak_list
              for radius in morph_radius_list]
plot_detection.plot_ims([eqw, [[0,0],[0,0]]], ['',''])
plot_detection.plot_ims(ch_mask_list[:3], title_list[:3])

In [ ]:
# date_str = '2015_03_31__18_13'
date_str = '2015_07_16__17_12'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

percent_of_peak_list = [90, 90, 90]
morph_radius_list = [4,10,14]

ch_mask_list = [
    run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]
title_list = [f'{percent_of_peak:d}% of Peak | {radius:d}px Radius'
              for percent_of_peak in percent_of_peak_list
              for radius in morph_radius_list]
plot_detection.plot_ims([eqw, [[0,0],[0,0]]], ['',''])
plot_detection.plot_ims(ch_mask_list[:3], title_list[:3])

In [ ]:
date_str = '2015_03_31__18_13'


raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

step = 1
morph_radius_lists = [list(np.arange(1,21,step))]
percent_of_peak_list = [100]

for morph_radius_list in morph_radius_lists:
    ch_mask_list = run_detection.get_CH_mask_list(eqw, percent_of_peak_list, morph_radius_list)
    
    area_percent_list = run_detection.get_area_percent_list(ch_mask_list)
    
    parameter_stats = run_detection.get_parameter_stats(area_percent_list)
    max_diff, cutoff, selected_parameter_num, pixel_percent_diffs = parameter_stats
    
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot()

    plot_detection.plot_pixel_percent_bars(
        ax, morph_radius_list, area_percent_list, max_diff, cutoff, selected_parameter_num,
        step, title='SE Disk Radius', unit='px', xlabel='SE Disk Radius (px)', thresh=False
    )

## Smoothness

### Brightness Width

In [ ]:
date_str = '2015_03_31__18_13'
percent_of_peak = 90
morph_radius = 15

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)

plot_detection.plot_sorted_ch_hists(eqw, ch_mask, apply_gradient=False,
                                    hist_stat='width')

In [ ]:
percent_of_peak = 90
morph_radius = 15
apply_gradient = False
hist_stat = 'median'
ascend_sort = False

for eqw_date in HE_DATE_LIST:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    
    ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    
    ranked_map = run_detection.get_ranked_map(
        eqw, ch_mask, apply_gradient, hist_stat, ascend_sort
    )

    euv = EUV_DICT[eqw_date]

    plot_detection.plot_ensemble_comparison(eqw, eqw_date, ranked_map, euv)

### Brightness Tail Width

In [ ]:
date_str = '2015_03_31__18_13'
percent_of_peak = 90
morph_radius = 15

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)

plot_detection.plot_sorted_ch_hists(eqw, ch_mask, apply_gradient=False,
                                    hist_stat='tail_width')

In [ ]:
percent_of_peak = 90
morph_radius = 15
apply_gradient = False
hist_stat = 'tail_width'

for eqw_date in HE_DATE_LIST:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    
    ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    ranked_map = run_detection.get_ranked_map(
        eqw, ch_mask, apply_gradient, hist_stat
    )

    euv = EUV_DICT[eqw_date]

    plot_detection.plot_ensemble_comparison(eqw, eqw_date, ranked_map, euv)

### Gradient Median

In [ ]:
date_str = '2015_03_31__18_13'
percent_of_peak = 90
morph_radius = 15

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)

plot_detection.plot_sorted_ch_hists(eqw, ch_mask, apply_gradient=True,
                                    hist_stat='median')

In [ ]:
percent_of_peak = 90
morph_radius = 15
apply_gradient = True
hist_stat = 'median'

for date_str in HE_DATE_LIST[-1:]:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    
    ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    ranked_map = run_detection.get_ranked_map(
        eqw, ch_mask, apply_gradient, hist_stat
    )

    euv = EUV_DICT[date_str]

    plot_detection.plot_ensemble_comparison(eqw, date_str, ranked_map, euv)

### Gradient Width

In [ ]:
date_str = '2015_03_31__18_13'
percent_of_peak = 90
morph_radius = 15

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)

plot_detection.plot_sorted_ch_hists(eqw, ch_mask, apply_gradient=True,
                                    hist_stat='width')

In [ ]:
percent_of_peak = 90
morph_radius = 15
apply_gradient = True
hist_stat = 'width'

for eqw_date in HE_DATE_LIST:
    raw_eqw = HE_FITS_DICT[date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    
    ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    ranked_map = run_detection.get_ranked_map(
        eqw, ch_mask, apply_gradient, hist_stat
    )

    euv = EUV_DICT[eqw_date]

    plot_detection.plot_ensemble_comparison(eqw, eqw_date, ranked_map, euv)

### Date Comparison

In [ ]:
percent_of_peak = 90
morph_radius = 15
eqw_date = HE_DATE_LIST[1]
apply_gradient_list = [False, False, True, True]
hist_stat_list = ['width', 'tail_width', 'median', 'width']

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

ch_mask = run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)

euv = EUV_DICT[eqw_date]
    
for apply_gradient, hist_stat in zip(apply_gradient_list, hist_stat_list):
    ranked_map = run_detection.get_ranked_map(
        eqw, ch_mask, apply_gradient, hist_stat
    )
    plot_detection.plot_ensemble_comparison(eqw, eqw_date, ranked_map, euv)

# Outcomes

## Varied Parameter CH Masks

In [ ]:
date_str = '2015_03_31__18_13'
raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
save_file = f'{SINGLE_MAPS_DIR}{date_str}_maps.npy'

Create CH Mask List (Expensive computation)

In [ ]:
# Lower threshold accepts more and lets morphology carry the load in selection and removal
thresh_step = 5
radius_step = 1
percent_of_peak_list = list(np.arange(70,106,thresh_step))
morph_radius_list = list(np.arange(6,21,radius_step))

# List of CHS masks for different files with varied parameters
all_ch_mask_list = [
    run_detection.get_ch_mask(eqw, percent_of_peak, morph_radius)
    for percent_of_peak, morph_radius
    in zip(percent_of_peak_list, morph_radius_list)
]

save_list = [date_str, percent_of_peak_list, morph_radius_list, all_ch_mask_list]
np.save(save_file, np.array(save_list, dtype=object), allow_pickle=True)

Load CH Mask List

In [ ]:
save_list = np.load(save_file, allow_pickle=True)
date_str = save_list[0]
percent_of_peak_list = save_list[1]
morph_radius_list = save_list[2]
all_ch_mask_list = save_list[3]

In [ ]:
title_list = [f'{percent_of_peak:d}% of Peak | {radius:d}px Radius'
              for percent_of_peak in percent_of_peak_list
              for radius in morph_radius_list]

## Area

In [ ]:
area_percent_list = run_detection.get_area_percent_list(all_ch_mask_list)

In [ ]:
heat_map_title = f'{date_str} Segmented Area Percentage'

plot_detection.plot_heat_map(
    area_percent_list, heat_map_title, percent_of_peak_list, morph_radius_list
)

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot()

ax.hist(area_percent_list[:-1], bins=30)
ax.set_title(f'{date_str} Segmented Area Bins', fontsize=20)
ax.set_xlabel('Area Percentage', fontsize=18)
ax.set_ylabel('Number of Masks in Area Bin', fontsize=18)

### Heat Map Bands

In [ ]:
lower_bound = 16
upper_bound = 18

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_detection.plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list
)

In [ ]:
lower_bound = 6.8
upper_bound = 7.3

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_detection.plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list
)

In [ ]:
lower_bound = 5.4
upper_bound = 6

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_detection.plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list
)

In [ ]:
lower_bound = 3.2
upper_bound = 3.3

heat_map_title = f'{date_str} {lower_bound}-{upper_bound}% Segmented Area'

plot_detection.plot_heat_map_band(
    area_percent_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list
)

## Hole Number

In [ ]:
num_ch_list = run_detection.get_num_CH_list(all_ch_mask_list)

In [ ]:
heat_map_title = f'{date_str} Segmented Hole Number'
color_scale = 'Aggrnyl'

plot_detection.plot_heat_map(
    num_ch_list, heat_map_title, percent_of_peak_list, morph_radius_list, color_scale
)

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot()

ax.hist(num_ch_list, bins=30, range=(0,25))
ax.set_title(f'{date_str} Segmented Hole Number Bins', fontsize=20)
ax.set_xlabel('Segemented Hole Number', fontsize=18)
ax.set_ylabel('Number of Masks in Hole Number Bin', fontsize=18)

### Heat Map Bands

In [ ]:
lower_bound = 17
upper_bound = 17

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Segmented Holes'
color_scale = 'Aggrnyl'

plot_detection.plot_heat_map_band(
    num_ch_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list, color_scale
)

In [ ]:
lower_bound = 11
upper_bound = 11

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Segmented Holes'
color_scale = 'Aggrnyl'

plot_detection.plot_heat_map_band(
    num_ch_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list, color_scale
)

In [ ]:
lower_bound = 9
upper_bound = 9

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Segmented Holes'
color_scale = 'Aggrnyl'

plot_detection.plot_heat_map_band(
    num_ch_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list, color_scale
)

## Lower Tail Width

In [ ]:
lower_tail_width_list = run_detection.get_ch_lower_tail_width_list(eqw, all_ch_mask_list)

In [ ]:
heat_map_title = f'{date_str} Mean CH Tail Width'
color_scale = 'ice'

plot_detection.plot_heat_map(
    lower_tail_width_list, heat_map_title, percent_of_peak_list, morph_radius_list, color_scale
)

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot()

ax.hist(lower_tail_width_list, bins=30, range=(10,40))
ax.set_title(f'{date_str} Mean CH Tail Width Bins', fontsize=20)
ax.set_xlabel('Mean CH Tail Width', fontsize=18)
ax.set_ylabel('Number of Masks in Tail Width Bin', fontsize=18)

### Heat Map Bands

In [ ]:
lower_bound = 29
upper_bound = 29.5

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Mean CH Tail Width'
color_scale = 'ice'

plot_detection.plot_heat_map_band(
    lower_tail_width_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list, color_scale
)

In [ ]:
lower_bound = 24.5
upper_bound = 25

heat_map_title = f'{date_str} {lower_bound}-{upper_bound} Mean CH Tail Width'
color_scale = 'ice'

plot_detection.plot_heat_map_band(
    lower_tail_width_list, heat_map_title, lower_bound, upper_bound,
    percent_of_peak_list, morph_radius_list, 
    eqw, all_ch_mask_list, title_list, color_scale
)

## Multiplied

In [ ]:
heat_map_title = f'{date_str} Multiplied Metrics'
color_scale = 'dense_r'

outcome_list = list(np.array(area_percent_list)*np.array(num_ch_list)*np.array(lower_tail_width_list))

plot_detection.plot_heat_map(
    outcome_list, heat_map_title, percent_of_peak_list, morph_radius_list, color_scale
)

# Ensemble

## Individual

### Sorted by Area

v0.2 Pre-Process

In [ ]:
date_str = '2012_05_07__17_59'
# date_str = '2013_04_17__18_39'
# date_str = '2014_06_26__14_19'
# date_str = '2015_03_31__18_13'
# date_str = '2015_07_16__17_12'


raw_eqw = HE_FITS_DICT[date_str][0]

# v0.2
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17]

ensemble_map, ch_masks, confidence_list = run_detection.get_ensemble_v0_2(
    eqw, percent_of_peak_list, morph_radius_list
)
plot_detection.plot_ensemble(
    eqw, ensemble_map, confidence_list, ch_masks
)

v0.3 Pre-Process

In [ ]:
date_str = '2012_06_22__18_13'
# date_str = '2014_06_26__14_19'
# date_str = '2015_03_31__18_13'
# date_str = '2015_07_16__17_12'

raw_eqw = HE_FITS_DICT[date_str][0]

# v0.3
eqw = pre_process_eqw_v0_3(raw_eqw)[0]
percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17]

ensemble_map, ch_masks, confidence_list = run_detection.get_ensemble_v0_2(
    eqw, percent_of_peak_list, morph_radius_list
)
plot_detection.plot_ensemble(
    eqw, ensemble_map, confidence_list, ch_masks
)

### Sorted by Smoothness

In [ ]:
# date_str = '2012_06_22__18_13'
# date_str = '2014_06_26__14_19'
date_str = '2015_03_31__18_13'
# date_str = '2015_07_16__17_12'

percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17]

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
ensemble_map, isolated_ch_im_list, confidence_list = run_detection.get_ensemble_v0_3(
    eqw, percent_of_peak_list, morph_radius_list
)
plot_detection.plot_ensemble(
    eqw, ensemble_map, confidence_list, isolated_ch_im_list
)

#### Percentage Rank

In [ ]:
# date_str = '2013_04_17__18_39'
# date_str = '2014_06_26__14_19'
date_str = '2015_03_31__18_13'
# date_str = '2015_07_16__17_12'

percent_of_peak_list = [80,80,90,100,100]
morph_radius_list = [13,17,15,13,17]

raw_eqw = HE_FITS_DICT[date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
ensemble_map, isolated_ch_ims, confidence_list = run_detection.get_smooth_ensemble(
    eqw, percent_of_peak_list, morph_radius_list, percent_rank=True
)
plot_detection.plot_ensemble(
    eqw, ensemble_map, confidence_list, isolated_ch_ims
)

## Outcomes

In [ ]:
from scipy import ndimage
from datetime import datetime
from matplotlib import colors, colormaps


oldcmp = colormaps['PuBuGn']
newcolors = oldcmp(np.linspace(0, 1, 256))
newcolors = newcolors[20:]

background = np.array([0, 0, 0, 1])
newcolors[:1, :] = background
confidence_cmap = colors.ListedColormap(newcolors)

Threshold by Confidence

In [ ]:
he_date_str = HE_DATE_LIST[52]
confidence_level = 90

raw_eqw = HE_FITS_DICT[he_date_str][0]
eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]

save_file = f'{ENSEMBLE_MAP_SAVE_DIR}{he_date_str}_ensemble_map.npy'
ensemble_map = run_detection.get_saved_ensemble_map(save_file)[-1]
confidence_map = np.where(ensemble_map >= confidence_level, ensemble_map, 0)

labeled_map, num_ch = ndimage.label(confidence_map)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

ax[0].set_title(he_date_str)
ax[0].imshow(eqw, cmap=plt.cm.gray)
ax[0].grid(False)

ax[1].imshow(confidence_map, cmap=plt.cm.magma)
ax[2].imshow(labeled_map, cmap=confidence_cmap)

In [ ]:
he_date_str_list = HE_DATE_LIST[48:62]
confidence_level_list = [0, 25, 50, 90]

num_ch_by_confidences_list = []


for he_date_str in he_date_str_list:
    save_file = f'{ENSEMBLE_MAP_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = run_detection.get_saved_ensemble_map(save_file)[-1]
    confidence_maps = [np.where(ensemble_map >= confidence_level, ensemble_map, 0)
                       for confidence_level in confidence_level_list]
    
    # List of number of CH detected by at given or greater confidence levels
    num_ch_by_confidences = [ndimage.label(confidence_map)[1]
                             for confidence_map in confidence_maps]
    num_ch_by_confidences = np.flip(num_ch_by_confidences)
    num_ch_by_confidences_list.append(num_ch_by_confidences)

In [ ]:
DICT_DATE_STR_FORMAT = '%Y_%m_%d__%H_%M'
OUTCOME_CMAP = colormaps['viridis']


def plot_outcome_vs_time(he_date_str_list, outcome_by_confidences_list, ylabel):
    """Plot outcome stacked plot vs time for ensemble maps.
    """
    datetime_list = [datetime.strptime(he_date_str, DICT_DATE_STR_FORMAT)
                     for he_date_str in he_date_str_list]
    
    # Array of differences in outcomes between confidence levels
    outcome_diff_by_confidences = np.array([
        np.append(outcome_by_confidences[0], np.diff(outcome_by_confidences))
        for outcome_by_confidences in outcome_by_confidences_list
    ])
    outcome_diff_by_confidences = outcome_diff_by_confidences.T
    
    # max_outcome = np.max(outcome_by_confidences_list)

    confidence_label_list = [f'{confidence_level}% Confidence'
                            for confidence_level in reversed(confidence_level_list)]

    color_list = OUTCOME_CMAP(np.linspace(0, 0.75, len(confidence_level_list)))

    plt.figure(figsize=(12,5))
    plt.stackplot(datetime_list, outcome_diff_by_confidences,
                labels=confidence_label_list, colors=color_list)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(ylabel)
    # plt.ylim([0, 1.05*max_outcome])
    plt.legend(reverse=True)


plot_outcome_vs_time(he_date_str_list, num_ch_by_confidences_list,
                     'Detected CH Number')

## Output

Create Ensemble Maps & Save to Files (Expensive computation)

In [ ]:
overwrite = False

# v0.2
# O X X
# X O O
# X O X
percent_of_peak_list = [80,80, 90, 100,100]
morph_radius_list = [15,17, 13, 13,17]

# date_idx = 48 # 32: 05/06-08   # 48: 05/26-28

if not os.path.isdir(ENSEMBLE_MAP_SAVE_DIR):
    os.makedirs(ENSEMBLE_MAP_SAVE_DIR)

# for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
for he_date_str in HE_DATE_LIST:
    
    # Optionally overwrite existing files
    save_file = f'{ENSEMBLE_MAP_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    if os.path.isfile(save_file) and not overwrite:
        print((f'He {he_date_str} ensemble map already exists.'))
        continue
    
    raw_eqw = HE_FITS_DICT[he_date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    # eqw = normalize(raw_eqw)[0]

    # ensemble_map = run_detection.get_ensemble(
    #     eqw, percent_of_peak_list, morph_radius_list
    # )[0]
    ensemble_map = run_detection.get_smooth_ensemble(
        eqw, percent_of_peak_list, morph_radius_list
    )[0]
    
    save_list = [he_date_str, percent_of_peak_list, morph_radius_list, ensemble_map]
    np.save(save_file, np.array(save_list, dtype=object), allow_pickle=True)
    print(f'{he_date_str} Ensemble Map Saved')

Save Ensemble Maps to Images

In [ ]:
output_dir = ENSEMBLE_MAPS_DIR + 'Preprocess_v0_3/2012_05_27/'
date_idx = 48 # 32: 05/06-08   # 48: 05/26-28


if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

for he_date_str in HE_DATE_LIST[date_idx:date_idx + 3]:
    
    raw_eqw = HE_FITS_DICT[he_date_str][0]
    eqw = run_detection.pre_process_eqw_v0_1(raw_eqw)[0]
    # eqw = normalize(raw_eqw)[0]
    
    save_file = f'{ENSEMBLE_MAP_SAVE_DIR}{he_date_str}_ensemble_map.npy'
    ensemble_map = run_detection.get_saved_ensemble_map(save_file)[-1]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax = axes.ravel()

    ax[0].set_title(he_date_str, fontsize=24)
    ax[0].imshow(eqw, cmap=plt.cm.gray)
    
    ax[1].imshow(ensemble_map, cmap=plt.cm.magma)

    plt.savefig(f'{output_dir}{he_date_str}.jpg')
    plt.close(fig)

Write Images to Video

In [ ]:
output_dir = ENSEMBLE_MAPS_DIR + 'Preprocess_v0_3/2012_05_27/'

fps = 2

run_detection.write_ensemble_video(output_dir, fps)